In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='sakila',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

1) Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [2]:
db('''
SELECT
    name, SUM(amount) as receita
FROM
    payment
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY 
    category_id
ORDER BY
    receita DESC
    
''')

Executando query:
('Sports', Decimal('138295.47'))
('Animation', Decimal('137116.48'))
('Action', Decimal('130684.74'))
('Family', Decimal('129048.71'))
('Sci-Fi', Decimal('127768.37'))
('Drama', Decimal('122052.28'))
('Documentary', Decimal('122046.18'))
('Foreign', Decimal('120780.50'))
('Games', Decimal('114451.03'))
('New', Decimal('110441.67'))
('Comedy', Decimal('110384.27'))
('Children', Decimal('109727.41'))
('Classics', Decimal('108921.82'))
('Horror', Decimal('98382.66'))
('Travel', Decimal('97279.09'))
('Music', Decimal('95838.49'))


2) Liste os filmes e o numero de vezes em que foram alugados

In [3]:
db('''
SELECT
    title, COUNT(rental_id) as vezes_alugados
FROM 
    film
    INNER JOIN inventory USING (film_id)
    INNER JOIN rental USING (inventory_id)
GROUP BY
    film_id
ORDER BY
    vezes_alugados DESC
    
''')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('FORWARD TEMPLE', 32)
('GRIT CLOCKWORK', 32)
('JUGGLER HARDLY', 32)
('RIDGEMONT SUBMARINE', 32)
('SCALAWAG DUCK', 32)
('APACHE DIVINE', 31)
('GOODFELLAS SALUTE', 31)
('HOBBIT ALIEN', 31)
('NETWORK PEAK', 31)
('ROBBERS JOON', 31)
('RUSH GOODFELLAS', 31)
('TIMBERLAND SKY', 31)
('WIFE TURN', 31)
('ZORRO ARK', 31)
('BUTTERFLY CHOCOLAT', 30)
('CAT CONEHEADS', 30)
('DOGMA FAMILY', 30)
('ENGLISH BULWORTH', 30)
('FROST HEAD', 30)
('GRAFFITI LOVE', 30)
('HARRY IDAHO', 30)
('IDOLS SNATCHERS', 30)
('MARRIED GO', 30)
('MASSACRE USUAL', 30)
('MUSCLE BRIGHT', 30)
('PULP BEVERLY', 30)
('RUGRATS SHAKESPEARE', 30)
('SHOCK CABIN', 30)
('SUSPECTS QUILLS', 30)
('WITCHES PANIC', 30)
('BINGO TALENTED', 29)
('BOOGIE AMELIE', 29)
('CONFIDENTIAL INTERVIEW', 29)
('DEER VIRGINIAN', 29)
('ENEMY ODDS', 29)
('FAMILY SWEET', 29)
('GLEAMING JAWBREAKER', 29)
('GREATEST NORTH', 29)
('MOON BUNCH', 29)
('STORM HAPPINESS', 29)
('SWEETHEARTS SUSPECTS', 

3) Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme.

In [4]:
#primeiro vamos contar todos os filmes da base
db('''
SELECT 
    COUNT(film_id) INTO @cnt_film
FROM
    film
''')

Executando query:


In [5]:
#depois vamos contar quantos aluguéis temos na base de dados

db('''
SELECT
    COUNT(rental_id) INTO @cnt_rental
FROM
    rental
''')

Executando query:


In [6]:
# Agora vamos calcular a media dividind o numero de alugueis por filme

db('''
SELECT
    @cnt_rental/ @cnt_film INTO @avg_rental_per_film
''')

Executando query:


In [7]:
#Conferindo o resultado:

db('SELECT @avg_rental_per_film')

Executando query:
(Decimal('16.044000000'),)


In [8]:
db('''
SELECT
    title, COUNT(rental_id) as vezes_alugados
FROM 
    film
    INNER JOIN inventory USING (film_id)
    INNER JOIN rental USING (inventory_id)
GROUP BY
    film_id
HAVING    
    vezes_alugados > @avg_rental_per_film
ORDER BY
    vezes_alugados DESC
    
''')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('FORWARD TEMPLE', 32)
('GRIT CLOCKWORK', 32)
('JUGGLER HARDLY', 32)
('RIDGEMONT SUBMARINE', 32)
('SCALAWAG DUCK', 32)
('APACHE DIVINE', 31)
('GOODFELLAS SALUTE', 31)
('HOBBIT ALIEN', 31)
('NETWORK PEAK', 31)
('ROBBERS JOON', 31)
('RUSH GOODFELLAS', 31)
('TIMBERLAND SKY', 31)
('WIFE TURN', 31)
('ZORRO ARK', 31)
('BUTTERFLY CHOCOLAT', 30)
('CAT CONEHEADS', 30)
('DOGMA FAMILY', 30)
('ENGLISH BULWORTH', 30)
('FROST HEAD', 30)
('GRAFFITI LOVE', 30)
('HARRY IDAHO', 30)
('IDOLS SNATCHERS', 30)
('MARRIED GO', 30)
('MASSACRE USUAL', 30)
('MUSCLE BRIGHT', 30)
('PULP BEVERLY', 30)
('RUGRATS SHAKESPEARE', 30)
('SHOCK CABIN', 30)
('SUSPECTS QUILLS', 30)
('WITCHES PANIC', 30)
('BINGO TALENTED', 29)
('BOOGIE AMELIE', 29)
('CONFIDENTIAL INTERVIEW', 29)
('DEER VIRGINIAN', 29)
('ENEMY ODDS', 29)
('FAMILY SWEET', 29)
('GLEAMING JAWBREAKER', 29)
('GREATEST NORTH', 29)
('MOON BUNCH', 29)
('STORM HAPPINESS', 29)
('SWEETHEARTS SUSPECTS', 

Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

4) Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view movie_count

In [9]:
#Quantas vezes foram alugados mesmo para filmes que nunca foram alugados: com cnt igual a 0
#Todos os filmes serão presentes mesmo que não haja [cópia física em lugar nenhum = inventory]
# e mesmo que nunca tenha sido alugado na vida
#pode ter quantos nulos for, ele conta como 0
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Executando query:
Executando query:


In [10]:
db('''
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


In [11]:
db('''
SELECT
    cnt, title
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
(7, 'COWBOY DOOM')


5) Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'


In [12]:
db('''
SELECT
    MIN(inventory_id) INTO @inventory_id
FROM
    film
    INNER JOIN inventory USING(film_id)
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:


In [13]:
db('''
SELECT
    customer_id INTO @customer_id
FROM
    customer
WHERE
    first_name = 'JESSIE'
    AND last_name = 'BANKS'
''')

Executando query:


In [14]:
db('''
SELECT
    staff_id INTO @staff_id
FROM
    staff
WHERE
    first_name = 'Jon'
    AND last_name = 'Stephens'
''')

Executando query:


In [15]:
db('''
INSERT INTO rental (
    rental_date, 
    inventory_id, 
    customer_id, 
    return_date, 
    staff_id
)
VALUES 
(
    "2019-01-01",
    @inventory_id,
    @customer_id,
    "2019-01-08",
    @staff_id
)''')

Executando query:


In [16]:
db('''
SELECT
    title, cnt
FROM 
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 8)


In [17]:
connection.rollback()

In [18]:
db('''
SELECT
    title, cnt
FROM 
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 7)


6) Crie uma tabela temporária max_duration que contém a duração máxima de filme para cada categoria

In [19]:
db('''DROP TABLE IF EXISTS max_duration''')

db('''
CREATE TEMPORARY TABLE max_duration
    SELECT
        MAX(length) as duration, name, category_id
    FROM
        film
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        category_id
    ORDER BY
        duration DESC

''')

Executando query:
Executando query:


In [20]:
db('SELECT * FROM max_duration')

Executando query:
(185, 'Action', 1)
(185, 'Animation', 2)
(185, 'Comedy', 5)
(185, 'Games', 10)
(185, 'Music', 12)
(185, 'Sci-Fi', 14)
(185, 'Travel', 16)
(184, 'Classics', 4)
(184, 'Family', 8)
(184, 'Foreign', 9)
(184, 'Sports', 15)
(183, 'Documentary', 6)
(183, 'New', 13)
(181, 'Drama', 7)
(181, 'Horror', 11)
(178, 'Children', 3)


7) Agora use a tabela temporária para construir uma tabela com as categorias e seus respectivos filmes mais longos:

In [21]:
db('''
SELECT 
    title, length, name
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN max_duration USING (category_id)
WHERE
    length = duration
''')

Executando query:
('DARN FORRESTER', 185, 'Action')
('WORST BANGER', 185, 'Action')
('GANGS PRIDE', 185, 'Animation')
('POND SEATTLE', 185, 'Animation')
('CONTROL ANTHEM', 185, 'Comedy')
('CHICAGO NORTH', 185, 'Games')
('HOME PITY', 185, 'Music')
('SOLDIERS EVOLUTION', 185, 'Sci-Fi')
('MUSCLE BRIGHT', 185, 'Travel')
('SWEET BROTHERHOOD', 185, 'Travel')
('CONSPIRACY SPIRIT', 184, 'Classics')
('KING EVOLUTION', 184, 'Family')
('CRYSTAL BREAKING', 184, 'Foreign')
('SORORITY QUEEN', 184, 'Foreign')
('SMOOCHY CONTROL', 184, 'Sports')
('WIFE TURN', 183, 'Documentary')
('YOUNG LANGUAGE', 183, 'Documentary')
('FRONTIER CABIN', 183, 'New')
('JACKET FRISCO', 181, 'Drama')
('ANALYZE HOOSIERS', 181, 'Horror')
('LOVE SUICIDES', 181, 'Horror')
('FURY MURDER', 178, 'Children')
('WRONG BEHAVIOR', 178, 'Children')


Delete a tabela temporária

In [22]:
db('DROP TABLE IF EXISTS max_duration')

Executando query:


8) Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [23]:
db('DROP TABLE IF EXISTS films_por_categoria')

db('''
CREATE TEMPORARY TABLE films_por_categoria
    SELECT
        actor_id, first_name, last_name, CONCAT(name, ": ", GROUP_CONCAT(title SEPARATOR ", ")) as films 
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
        INNER JOIN film USING (film_id)
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        actor_id, category_id
''')

Executando query:
Executando query:


In [24]:
db('SELECT * FROM films_por_categoria')

Executando query:
(1, 'PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS')
(1, 'PENELOPE', 'GUINESS', 'Children: LANGUAGE COWBOY')
(1, 'PENELOPE', 'GUINESS', 'Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT')
(1, 'PENELOPE', 'GUINESS', 'Comedy: VERTIGO NORTHWEST')
(1, 'PENELOPE', 'GUINESS', 'Documentary: ACADEMY DINOSAUR')
(1, 'PENELOPE', 'GUINESS', 'Family: KING EVOLUTION, SPLASH GUMP')
(1, 'PENELOPE', 'GUINESS', 'Foreign: MULHOLLAND BEAST')
(1, 'PENELOPE', 'GUINESS', 'Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI')
(1, 'PENELOPE', 'GUINESS', 'Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN')
(1, 'PENELOPE', 'GUINESS', 'Music: WIZARD COLDBLOODED')
(1, 'PENELOPE', 'GUINESS', 'New: ANGELS LIFE, OKLAHOMA JUMANJI')
(1, 'PENELOPE', 'GUINESS', 'Sci-Fi: CHEAPER CLYDE')
(1, 'PENELOPE', 'GUINESS', 'Sports: GLEAMING JAWBREAKER')
(2, 'NICK', 'WAHLBERG', 'Action: BULL SHAWSHANK')
(2, 'NICK', 'WAHLBERG', 'Animation: FIGHT JAWBREAKER')
(2, 'NICK', 'WAHLBERG', 'Children: JERSEY SASSY')
(2, 'N

(134, 'GENE', 'HOPKINS', 'Comedy: HEDWIG ALTER')
(134, 'GENE', 'HOPKINS', 'Documentary: DEEP CRUSADE')
(134, 'GENE', 'HOPKINS', 'Drama: PITY BOUND, TORQUE BOUND')
(134, 'GENE', 'HOPKINS', 'Family: CHISUM BEHAVIOR')
(134, 'GENE', 'HOPKINS', 'Games: NIGHTMARE CHILL, SUIT WALLS')
(134, 'GENE', 'HOPKINS', 'Music: CLUE GRAIL, ELF MURDER, VANISHING ROCKY')
(134, 'GENE', 'HOPKINS', 'New: MANNEQUIN WORST')
(134, 'GENE', 'HOPKINS', 'Sci-Fi: CHAINSAW UPTOWN, PANTHER REDS')
(134, 'GENE', 'HOPKINS', 'Travel: GUMP DATE')
(135, 'RITA', 'REYNOLDS', 'Animation: LUKE MUMMY')
(135, 'RITA', 'REYNOLDS', 'Children: MAKER GABLES')
(135, 'RITA', 'REYNOLDS', 'Classics: GILBERT PELICAN, JERK PAYCHECK')
(135, 'RITA', 'REYNOLDS', 'Comedy: SNATCH SLIPPER, TRAINSPOTTING STRANGERS')
(135, 'RITA', 'REYNOLDS', 'Documentary: NATIONAL STORY')
(135, 'RITA', 'REYNOLDS', 'Family: EARRING INSTINCT, INDIAN LOVE')
(135, 'RITA', 'REYNOLDS', 'Foreign: COMMAND DARLING, GENTLEMEN STAGE, IMPOSSIBLE PREJUDICE')
(135, 'RITA', 'REYN

In [25]:
db('''
SELECT
    MIN(first_name), last_name, GROUP_CONCAT(films SEPARATOR "; ") as all_films 
FROM
    films_por_categoria
GROUP BY
    actor_id

''')

Executando query:
('PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER')
('NICK', 'WAHLBERG', 'Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAIS

8) quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

In [26]:
#primeiro vamos ver as variáveis
db('''
SELECT 
    AVG(rental_rate), STDDEV(rental_rate) INTO @avg_rate, @stddev_rate 
FROM
    film;
''')

Executando query:


In [27]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


In [28]:
#Agora podemos selecionar os filmes caros!

db('''
SELECT
    title, rental_rate
FROM
    film
WHERE
    rental_rate >  @avg_rate + @stddev_rate
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))
('ATTACKS HATE', Decimal('4.99'))
('ATTRACTION NEWTON', Decimal('4.99'))
('AUTUMN CROW', Decimal('4.99'))
('BABY HALL', Decimal('4.99'))
('BACKLASH UNDEFEATED', Decimal('4.99'))
('BEAST HUNCHBACK', Decimal('4.99'))
('BEAUTY GREASE', Decimal('4.99'))
('BEHAVIOR RUNAWAY', Decimal('4.99'))
('BETRAYED REAR', Decimal('4.99'))
('BIKINI BORROWERS', Decimal('4.99'))
('BILKO ANONYMOUS', Decimal('4.99'))
('BIRCH ANTITRUST', Decimal('4.99'))
('BIRD INDEPENDENCE', Decimal('4.99'))
('BIRDS PERDITION', Decimal('4.99'))
('BLINDNESS GUN', Decimal('4.99'))
('BOILED DARES', Decimal('4.99'))
('BOOGIE AMELIE', Decimal('4.99')

9) Use variáveis temporárias para encontrar o ator que mais participou de filmes.

In [29]:
db('''
SELECT
    first_name, last_name, COUNT(film_id) as qtd_filmes
FROM
    actor
    LEFT OUTER JOIN film_actor USING (actor_id)
    LEFT OUTER JOIN film USING (film_id)
GROUP BY
    actor_id
ORDER BY
    qtd_filmes DESC


''')

Executando query:
('GINA', 'DEGENERES', 42)
('WALTER', 'TORN', 41)
('MARY', 'KEITEL', 40)
('MATTHEW', 'CARREY', 39)
('SANDRA', 'KILMER', 37)
('SCARLETT', 'DAMON', 36)
('UMA', 'WOOD', 35)
('VAL', 'BOLGER', 35)
('HENRY', 'BERRY', 35)
('GROUCHO', 'DUNST', 35)
('ANGELA', 'WITHERSPOON', 35)
('VIVIEN', 'BASINGER', 35)
('ANGELA', 'HUDSON', 34)
('KIRSTEN', 'AKROYD', 34)
('SIDNEY', 'CROWE', 34)
('WARREN', 'NOLTE', 34)
('JAYNE', 'NOLTE', 34)
('RIP', 'CRAWFORD', 33)
('JULIA', 'MCQUEEN', 33)
('BEN', 'WILLIS', 33)
('SEAN', 'GUINESS', 33)
('KENNETH', 'TORN', 33)
('SUSAN', 'DAVIS', 33)
('CAMERON', 'ZELLWEGER', 33)
('RENEE', 'TRACY', 33)
('KEVIN', 'GARLAND', 33)
('EWAN', 'GOODING', 33)
('ALBERT', 'JOHANSSON', 33)
('RENEE', 'BALL', 33)
('REESE', 'WEST', 33)
('HELEN', 'VOIGHT', 32)
('REESE', 'KILMER', 32)
('NATALIE', 'HOPKINS', 32)
('CHRISTIAN', 'AKROYD', 32)
('WARREN', 'JACKMAN', 32)
('JULIANNE', 'DENCH', 32)
('ED', 'MANSFIELD', 32)
('WHOOPI', 'HURT', 32)
('GRETA', 'MALDEN', 32)
('HARVEY', 'HOPE', 32)


10) Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [32]:
db('''
SELECT
    actor_id, first_name, last_name, COUNT(film_id) as qtd_filmes
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
GROUP BY
    actor_id
ORDER BY
    qtd_filmes DESC
LIMIT
    3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de actor_id desses atores:

In [33]:
db('DROP TABLE IF EXISTS temp_pop_actors')


db('''
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT
        actor_id, first_name, last_name, COUNT(film_id) as qtd_filmes
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
        INNER JOIN film USING (film_id)
    GROUP BY
        actor_id
    ORDER BY
        qtd_filmes DESC
    LIMIT
        3
''')

db('SELECT * FROM temp_pop_actors')

Executando query:
Executando query:
Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [40]:
#    temp_pop_actors.actor_id = film_actor.actor_id

db('''
SELECT
    first_name, last_name, title
FROM
    temp_pop_actors
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
WHERE
    temp_pop_actors.actor_id = film_actor.actor_id
''')

Executando query:
('GINA', 'DEGENERES', 'BED HIGHBALL')
('GINA', 'DEGENERES', 'CALENDAR GUNFIGHT')
('GINA', 'DEGENERES', 'CHAMBER ITALIAN')
('GINA', 'DEGENERES', 'CHAPLIN LICENSE')
('GINA', 'DEGENERES', 'CHARIOTS CONSPIRACY')
('GINA', 'DEGENERES', 'CLUELESS BUCKET')
('GINA', 'DEGENERES', 'COLDBLOODED DARLING')
('GINA', 'DEGENERES', 'CONEHEADS SMOOCHY')
('GINA', 'DEGENERES', 'DARKNESS WAR')
('GINA', 'DEGENERES', 'DEER VIRGINIAN')
('GINA', 'DEGENERES', 'DOGMA FAMILY')
('GINA', 'DEGENERES', 'ELEPHANT TROJAN')
('GINA', 'DEGENERES', 'EXCITEMENT EVE')
('GINA', 'DEGENERES', 'FRISCO FORREST')
('GINA', 'DEGENERES', 'GANDHI KWAI')
('GINA', 'DEGENERES', 'GOODFELLAS SALUTE')
('GINA', 'DEGENERES', 'GUNFIGHT MOON')
('GINA', 'DEGENERES', 'HALL CASSIDY')
('GINA', 'DEGENERES', 'HEARTBREAKERS BRIGHT')
('GINA', 'DEGENERES', 'HOOK CHARIOTS')
('GINA', 'DEGENERES', 'HYDE DOCTOR')
('GINA', 'DEGENERES', 'IMPACT ALADDIN')
('GINA', 'DEGENERES', 'INDIAN LOVE')
('GINA', 'DEGENERES', 'INTRIGUE WORST')
('GINA', 'DE

In [42]:
#COM O USO DO IN:

db('''
SELECT
    first_name, last_name,  title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')


Executando query:
('GINA', 'DEGENERES', 'BED HIGHBALL')
('GINA', 'DEGENERES', 'CALENDAR GUNFIGHT')
('GINA', 'DEGENERES', 'CHAMBER ITALIAN')
('GINA', 'DEGENERES', 'CHAPLIN LICENSE')
('GINA', 'DEGENERES', 'CHARIOTS CONSPIRACY')
('GINA', 'DEGENERES', 'CLUELESS BUCKET')
('GINA', 'DEGENERES', 'COLDBLOODED DARLING')
('GINA', 'DEGENERES', 'CONEHEADS SMOOCHY')
('GINA', 'DEGENERES', 'DARKNESS WAR')
('GINA', 'DEGENERES', 'DEER VIRGINIAN')
('GINA', 'DEGENERES', 'DOGMA FAMILY')
('GINA', 'DEGENERES', 'ELEPHANT TROJAN')
('GINA', 'DEGENERES', 'EXCITEMENT EVE')
('GINA', 'DEGENERES', 'FRISCO FORREST')
('GINA', 'DEGENERES', 'GANDHI KWAI')
('GINA', 'DEGENERES', 'GOODFELLAS SALUTE')
('GINA', 'DEGENERES', 'GUNFIGHT MOON')
('GINA', 'DEGENERES', 'HALL CASSIDY')
('GINA', 'DEGENERES', 'HEARTBREAKERS BRIGHT')
('GINA', 'DEGENERES', 'HOOK CHARIOTS')
('GINA', 'DEGENERES', 'HYDE DOCTOR')
('GINA', 'DEGENERES', 'IMPACT ALADDIN')
('GINA', 'DEGENERES', 'INDIAN LOVE')
('GINA', 'DEGENERES', 'INTRIGUE WORST')
('GINA', 'DE

In [43]:
db('DROP TABLE temp_pop_actors')

Executando query:


11) Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

In [50]:
#Mais receita: mais amount
# Primeiro pega os filmes mais rentáveis e faz uma temp table e depois pega os atores desses filmes

db('DROP TABLE IF EXISTS top_3_filmes')

db('''
CREATE TEMPORARY TABLE top_3_filmes
    SELECT
        film_id, title, SUM(amount) as preco_tot_filme
    FROM
        film
        INNER JOIN inventory USING (film_id)
        INNER JOIN rental USING (inventory_id)
        INNER JOIN payment USING (rental_id)
    GROUP BY
        film_id
    ORDER BY
        preco_tot_filme DESC
    LIMIT
        3
''')

db('SELECT * FROM top_3_filmes')

Executando query:
Executando query:
Executando query:
(879, 'TELEGRAPH VOYAGE', Decimal('231.73'))
(973, 'WIFE TURN', Decimal('223.69'))
(1000, 'ZORRO ARK', Decimal('214.69'))


In [51]:
db('''
SELECT
    first_name, last_name, title
FROM
    top_3_filmes
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
''')

Executando query:
('WOODY', 'HOFFMAN', 'TELEGRAPH VOYAGE')
('CARMEN', 'HUNT', 'TELEGRAPH VOYAGE')
('GINA', 'DEGENERES', 'TELEGRAPH VOYAGE')
('LUCILLE', 'DEE', 'TELEGRAPH VOYAGE')
('VIVIEN', 'BASINGER', 'TELEGRAPH VOYAGE')
('MICHAEL', 'BENING', 'TELEGRAPH VOYAGE')
('THORA', 'TEMPLE', 'TELEGRAPH VOYAGE')
('WOODY', 'HOFFMAN', 'WIFE TURN')
('JULIA', 'BARRYMORE', 'WIFE TURN')
('GINA', 'DEGENERES', 'WIFE TURN')
('CAMERON', 'ZELLWEGER', 'WIFE TURN')
('GRETA', 'MALDEN', 'WIFE TURN')
('MICHAEL', 'BOLGER', 'WIFE TURN')
('IAN', 'TANDY', 'ZORRO ARK')
('NICK', 'DEGENERES', 'ZORRO ARK')
('LISA', 'MONROE', 'ZORRO ARK')


In [54]:
# OUUU USANDO O IN:

db('''
SELECT
    DISTINCT(first_name), last_name
FROM
    film_actor
    INNER JOIN actor USING (actor_id)
WHERE
    film_id IN (SELECT film_id FROM top_3_filmes);
''')

Executando query:
('WOODY', 'HOFFMAN')
('CARMEN', 'HUNT')
('GINA', 'DEGENERES')
('LUCILLE', 'DEE')
('VIVIEN', 'BASINGER')
('MICHAEL', 'BENING')
('THORA', 'TEMPLE')
('JULIA', 'BARRYMORE')
('CAMERON', 'ZELLWEGER')
('GRETA', 'MALDEN')
('MICHAEL', 'BOLGER')
('IAN', 'TANDY')
('NICK', 'DEGENERES')
('LISA', 'MONROE')


12) Vamos refazer a atividade dos filmes de atores populares, usando subqueries.

Temos um problema: o MySQL não suporta LIMIT em subqueries com o operador IN. Vamos investigar isso mais de perto.

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

SOLUÇÃO : FAZER OUTRA SUBQUERY DO JEITO QUE FICOU:

select * from test 
where code_ver IN (select * from (select DISTINCT code_ver 
                      from test 
                     where code_ver NOT LIKE '%DevBld%' 
                     ORDER by date DESC LIMIT 10) as t1);
                     
FONTE: https://stackoverflow.com/questions/12810346/alternative-to-using-limit-keyword-in-a-subquery-in-mysql 

In [66]:
db('''
SELECT
    first_name, last_name, title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE 
    film.film_id IN
    (SELECT * FROM (SELECT
            film_id
        FROM
            film
            INNER JOIN inventory USING (film_id)
            INNER JOIN rental USING (inventory_id)
            INNER JOIN payment USING (rental_id)
        GROUP BY
            film_id
        ORDER BY
            SUM(amount) DESC
        LIMIT 3
        ) as SUBQUERY
    )

''')

Executando query:
('WOODY', 'HOFFMAN', 'TELEGRAPH VOYAGE')
('CARMEN', 'HUNT', 'TELEGRAPH VOYAGE')
('GINA', 'DEGENERES', 'TELEGRAPH VOYAGE')
('LUCILLE', 'DEE', 'TELEGRAPH VOYAGE')
('VIVIEN', 'BASINGER', 'TELEGRAPH VOYAGE')
('MICHAEL', 'BENING', 'TELEGRAPH VOYAGE')
('THORA', 'TEMPLE', 'TELEGRAPH VOYAGE')
('WOODY', 'HOFFMAN', 'WIFE TURN')
('JULIA', 'BARRYMORE', 'WIFE TURN')
('GINA', 'DEGENERES', 'WIFE TURN')
('CAMERON', 'ZELLWEGER', 'WIFE TURN')
('GRETA', 'MALDEN', 'WIFE TURN')
('MICHAEL', 'BOLGER', 'WIFE TURN')
('IAN', 'TANDY', 'ZORRO ARK')
('NICK', 'DEGENERES', 'ZORRO ARK')
('LISA', 'MONROE', 'ZORRO ARK')


### UNION
Quando duas tabelas tem EXATAMENTE as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador UNION. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [90]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('''
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
''')

db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

# Criando uma tabela temporária com os nomes dos clientes

Executando query:
Executando query:
Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')


In [91]:
# Criar tabela temporária com o nome dos funcionários

db('DROP TABLE IF EXISTS nomes_staff')

db('''
CREATE TEMPORARY TABLE nomes_staff
    SELECT first_name, last_name FROM staff
''')

db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

Executando query:
Executando query:
Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')


In [92]:
db('DROP TABLE IF EXISTS nomes_all')

db('''
CREATE TEMPORARY TABLE nomes_all
    (SELECT * FROM nomes_staff) UNION (SELECT * FROM nomes_clientes)
''')

db('SELECT * FROM nomes_all ')


Executando query:
Executando query:
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')
('JENNIFER', 'DAVIS')
('MARIA', 'MILLER')
('SUSAN', 'WILSON')
('MARGARET', 'MOORE')
('DOROTHY', 'TAYLOR')
('LISA', 'ANDERSON')
('NANCY', 'THOMAS')
('KAREN', 'JACKSON')
('BETTY', 'WHITE')
('HELEN', 'HARRIS')
('SANDRA', 'MARTIN')
('DONNA', 'THOMPSON')
('CAROL', 'GARCIA')
('RUTH', 'MARTINEZ')
('SHARON', 'ROBINSON')
('MICHELLE', 'CLARK')
('LAURA', 'RODRIGUEZ')
('SARAH', 'LEWIS')
('KIMBERLY', 'LEE')
('DEBORAH', 'WALKER')
('JESSICA', 'HALL')
('SHIRLEY', 'ALLEN')
('CYNTHIA', 'YOUNG')
('ANGELA', 'HERNANDEZ')
('MELISSA', 'KING')
('BRENDA', 'WRIGHT')
('AMY', 'LOPEZ')
('ANNA', 'HILL')
('REBECCA', 'SCOTT')
('VIRGINIA', 'GREEN')
('KATHLEEN', 'ADAMS')
('PAMELA', 'BAKER')
('MARTHA', 'GONZALEZ')
('DEBRA', 'NELSON')
('AMANDA', 'CARTER')
('STEPHANIE', 'MITCHELL')
('CAROLYN', 'PEREZ')
('CHRISTINE', 'ROBERT

In [93]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('DROP TABLE IF EXISTS nomes_staff')
db('DROP TABLE IF EXISTS nomes_all')

Executando query:
Executando query:
Executando query:


13) Refaça o exemplo acima mas use subqueries ao invés de temp tables.

In [94]:
db('''
    (SELECT * FROM (SELECT first_name, last_name FROM customer) as sub1) UNION
    (SELECT * FROM (SELECT first_name, last_name FROM staff) as sub2)
''')


Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')
('JENNIFER', 'DAVIS')
('MARIA', 'MILLER')
('SUSAN', 'WILSON')
('MARGARET', 'MOORE')
('DOROTHY', 'TAYLOR')
('LISA', 'ANDERSON')
('NANCY', 'THOMAS')
('KAREN', 'JACKSON')
('BETTY', 'WHITE')
('HELEN', 'HARRIS')
('SANDRA', 'MARTIN')
('DONNA', 'THOMPSON')
('CAROL', 'GARCIA')
('RUTH', 'MARTINEZ')
('SHARON', 'ROBINSON')
('MICHELLE', 'CLARK')
('LAURA', 'RODRIGUEZ')
('SARAH', 'LEWIS')
('KIMBERLY', 'LEE')
('DEBORAH', 'WALKER')
('JESSICA', 'HALL')
('SHIRLEY', 'ALLEN')
('CYNTHIA', 'YOUNG')
('ANGELA', 'HERNANDEZ')
('MELISSA', 'KING')
('BRENDA', 'WRIGHT')
('AMY', 'LOPEZ')
('ANNA', 'HILL')
('REBECCA', 'SCOTT')
('VIRGINIA', 'GREEN')
('KATHLEEN', 'ADAMS')
('PAMELA', 'BAKER')
('MARTHA', 'GONZALEZ')
('DEBRA', 'NELSON')
('AMANDA', 'CARTER')
('STEPHANIE', 'MITCHELL')
('CAROLYN', 'PEREZ')
('CHRISTINE', 'ROBERTS')
('MARIE', 'TURNER')
('JANET', 'PHILLIPS')
('CATHERINE', 'CAMPBELL')
('FR

### Desafio 

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

In [98]:
# Primeiro: vamos ver os atores daquele filme. Group por filme
# Depois vamos ver as letras do filme e os atores. 

db('''
SELECT 
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    LEFT(title, 1) = LEFT(first_name, 1)    
GROUP BY
    film_id
HAVING
    COUNT(actor_id) > 2
''')

Executando query:
('CROW GREASE',)
('JEDI BENEATH',)
('SUBMARINE BED',)


Liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

In [102]:
# Primeiro: listar todos os filmes com o ator DAN_HARRIS

db('DROP TABLE IF EXISTS filmes_Dan')

db('''
CREATE TEMPORARY TABLE filmes_Dan
    SELECT
        film_id, title
    FROM 
        film
        INNER JOIN film_actor USING (film_id)
        INNER JOIN actor USING (actor_id)
    WHERE
        first_name = 'Dan'
        AND last_name = 'Harris'
''')


db('SELECT * FROM filmes_Dan')

Executando query:
Executando query:
Executando query:
(63, 'BEDAZZLED MARRIED')
(87, 'BOONDOCK BALLROOM')
(226, 'DESTINY SATURDAY')
(236, 'DIVINE RESURRECTION')
(298, 'EYES DRIVING')
(307, 'FELLOWSHIP AUTUMN')
(354, 'GHOST GROUNDHOG')
(383, 'GROOVE FICTION')
(417, 'HILLS NEIGHBORS')
(421, 'HOLIDAY GAMES')
(457, 'INDEPENDENCE HOTEL')
(462, 'INSIDER ARIZONA')
(474, 'JADE BUNCH')
(521, 'LIES TREATMENT')
(593, 'MONTEREY LABYRINTH')
(728, 'REUNION WITCHES')
(750, 'RUN PACIFIC')
(769, 'SCHOOL JACKET')
(781, 'SEVEN SWARM')
(795, 'SIEGE MADRE')
(844, 'STEERS ARMAGEDDON')
(851, 'STRAIGHT HOURS')
(862, 'SUMMER SCARFACE')
(868, 'SUPERFLY TRIP')
(892, 'TITANIC BOONDOCK')
(893, 'TITANS JERK')
(936, 'VANISHING ROCKY')
(965, 'WATERSHIP FRONTIER')


In [114]:
db('''
SELECT
    first_name, last_name, customer_id, filmes_Dan.film_id
FROM
    customer
    LEFT OUTER JOIN rental USING (customer_id)
    LEFT OUTER JOIN inventory USING (inventory_id)
    LEFT OUTER JOIN filmes_Dan USING (film_id)
WHERE
    filmes_Dan.film_id IS NULL
''')

Executando query:
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('MARY', 'SMITH', 1, None)
('PATRICIA', 'JOHNSON', 2, None)
('PATRICIA', 'JOHNSON', 2, None)
('PATRICIA', 'JOHNSON', 2, None)
('PATRICIA', 'JOHNS

('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('DORIS', 'REED', 55, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLORIA', 'COOK', 56, None)
('GLO

('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ETHEL', 'WEBB', 125, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SIMPSON', 126, None)
('ELLEN', 'SI

('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('ANA', 'BRADLEY', 181, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182, None)
('RENEE', 'LANE', 182,

('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('NAOMI', 'JENNINGS', 274, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE', 'BARNETT', 275, None)
('CAROLE',

('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('FRED', 'WHEAT', 369, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUONG', 370, None)
('WAYNE', 'TRUO

('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('TRAVIS', 'ESTEP', 417, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418, None)
('JEFF', 'EAST', 418,

('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('GLEN', 'TALBERT', 486, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, None)
('HECTOR', 'POINDEXTER', 487, N

('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('JIMMIE', 'EGGLESTON', 558, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EVERETT', 'BANDA', 559, None)
('EV

In [115]:
db('''
SELECT
    COUNT(DISTINCT customer_id)
FROM
    customer
    LEFT OUTER JOIN rental USING (customer_id)
    LEFT OUTER JOIN inventory USING (inventory_id)
    LEFT OUTER JOIN filmes_Dan USING (film_id)
WHERE
    filmes_Dan.film_id IS NULL
''')

Executando query:
(599,)


In [116]:
connection.close()